In [1]:
# Baixando e Instalando os modelos e pacotes necessários
%pip install transformers einops accelerate bitsandbytes

# Transformers: Processamento de linguagem natural, classificação, tradução e geraão de texto

# Einops: Manipulação de tensores (arrays multidimensionais)

# Accelerate: Treinamento distribuído e paralelo, reduz a complexidade do treinamento

# Bitsandbytes: Manipulação de bits e bytes, manupulação de arquivos binários e serialização

Note: you may need to restart the kernel to use updated packages.


In [2]:
# AutoModelForCausalLM: Modelo de linguagem causal, geração de texto
# AutoTokenizer: Tokenizador de texto, transforma texto em tokens
# pipeline: Processamento de texto, tokenização, geração de texto, tradução, classificação, etc
# BitsAndBytesConfig: Configuração para manipulação de bits e bytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [3]:
# torch: Biblioteca de aprendizado de máquina, manipulação de tensores, cálculos matemáticos

import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda:0'

In [4]:
# Configuração do modelo e tokenizador estatico para geração de texto
torch.random.manual_seed(42)

In [5]:
%pip install python-dotenv

import os
from dotenv import load_dotenv

load_dotenv()

# Guardar token
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

Note: you may need to restart the kernel to use updated packages.


In [13]:
# libera memoria da GPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
# torch.cuda.reset_max_memory_allocated() - obsoleto
# torch.cuda.reset_max_memory_cached() - obsoleto

In [14]:
id_modelo = "microsoft/Phi-3-mini-4k-instruct"

In [15]:
%pip install tiktoken

# Definindo a variável de ambiente para evitar fragmentação 
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Desabilita o paralelismo na biblioteca tokenizers para evitar problemas de deadlock
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)  # Ativa a quantização para 8 bits


# criando o modelo
model = AutoModelForCausalLM.from_pretrained(
    id_modelo,
    # device_map = 'balanced', # Usando a GPU/CPU automaticamente 
    device_map = 'cuda', # Usando a GPU
    # torch_dtype = 'auto', # Ajusta automaticamente os tipos de dados dos tensores do modelo
    torch_dtype = torch.float16, # Tipo de dado do modelo
    trust_remote_code = True, # Permite o carregamento personalizado de modelos
    # attn_implementation="eager", # metodo de implementação pro metodo de atenção
    quantization_config=quantization_config,
    )

Note: you may need to restart the kernel to use updated packages.


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# criando o tokenizador
tokenizer = AutoTokenizer.from_pretrained(
    id_modelo, 
    trust_remote_code = True # Permite o carregamento personalizado de modelos
    )

In [17]:
# criação do pipeline
pipe = pipeline(
    "text-generation", 
    model = model, 
    tokenizer = tokenizer
    )

Device set to use cuda


In [18]:
# parametros de geração de texto
generation_args = {
    "max_new_tokens": 500, # comprimento do texto de entrada
    "return_full_text": False, # determina se retorna todo o texto ou apenas o que foi gerado
    "temperature": 0.1, # controla a aleatoriedade da geração de texto
    "do_sample": True, # determina se a aleatoriedade é usada na geração de texto
    "num_beams": 3, # Número de feixes para busca 
    "early_stopping": True, # Pare a geração assim que uma sentença completa é gerada
    "truncation": True # Ativar a truncação explicitamente
}

In [19]:
prompt = "Explique o que é computação quântica"
output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [20]:
output

[{'generated_text': ' e como ela difere da computação clássica.\n\nComputação quântica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits binários (0 ou 1) para representar dados, a computação quântica usa qubits, que podem existir em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, potencialmente acelerando significativamente certos tipos de cálculos.\n\nUma das principais diferenças entre computação quântica e clássica é o conceito de entrelaçamento quântico. Quando dois qubits estão entrelaçados, o estado de um qubit depende do estado do outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos complexos de forma mais eficiente do que os computadores clássicos.\n\nOutra diferença chave é o uso de portas lógicas quânticas, que manipulam os estad

In [21]:
print(output[0]['generated_text'])

 e como ela difere da computação clássica.

Computação quântica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits binários (0 ou 1) para representar dados, a computação quântica usa qubits, que podem existir em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, potencialmente acelerando significativamente certos tipos de cálculos.

Uma das principais diferenças entre computação quântica e clássica é o conceito de entrelaçamento quântico. Quando dois qubits estão entrelaçados, o estado de um qubit depende do estado do outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos complexos de forma mais eficiente do que os computadores clássicos.

Outra diferença chave é o uso de portas lógicas quânticas, que manipulam os estados quânticos dos qubits. Es

In [22]:
prompt = "Quanto é 7 + 6 / 2"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

 - 3?

# Answer:Para resolver a expressão 7 + 6 / 2 - 3, precisamos seguir a ordem das operações, que é Parênteses, Expoentes, Multiplicação e Divisão (da esquerda para a direita), Adição e Subtração (da esquerda para a direita), também conhecida como PEMDAS.

1. Primeiro, realizamos a divisão: 6 / 2 = 3
2. Em seguida, realizamos a adição: 7 + 3 = 10
3. Por fim, realizamos a subtração: 10 - 3 = 7

Portanto, a resposta é 7.


In [26]:
prompt = "Quem foi a primeira pessoa a ir pro espaço?"

In [27]:
# template
template = """<|system|>
You are a helpful assistant.<|end>

<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [24]:
template

'<|system|>\nYou are a helpful assistant.<|end>\n\n<|user|>\n"Quanto é 7 + 6 / 2"<|end|>\n<|assistant|>'

In [28]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, a bordo da nave espacial Vostok 1, tornando-se um ícone mundial da exploração espacial.


In [33]:
prompt = "Qual é a capital do Brasil?" # @param {type: string}

sys_prompt = "Você é um assitente virtual que sempre responde em português do Brasil"

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

output = pipe(template, **generation_args)

print(template)

print(output[0]['generated_text'])

<|system|>
Você é um assitente virtual que sempre responde em português do Brasil<|end|>
<|user|>
"Qual é a capital do Brasil?"<|end|>
<|assistant|>
 A capital do Brasil é Brasília.


In [35]:
# formato de msg

prompt = "O que é IA?"

msg = [
    {
        "role": "system",
        "content": sys_prompt
    },
    {
        "role": "user",
        "content": prompt
    }
]

output = pipe(msg, **generation_args)

print(output[0]['generated_text'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 A IA, ou Inteligência Artificial, é um ramo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Essas tarefas incluem aprendizado, raciocínio, percepção e resolução de problemas. A IA pode ser dividida em duas categorias principais: IA fraca, também conhecida como inteligência artificial estreita, que é projetada para realizar tarefas específicas, e IA forte, ou inteligência artificial geral, que é capaz de realizar qualquer tarefa intelectual que um ser humano possa fazer. Exemplos de IA incluem assistentes virtuais como Siri e Alexa, sistemas de reconhecimento de fala, veículos autônomos e algoritmos de aprendizado de máquina.


In [36]:
# formato de msg

prompt = "Liste o nome de 10 cidades famosas da Europa"

msg = [
    {
        "role": "system",
        "content": sys_prompt
    },
    {
        "role": "user",
        "content": prompt
    }
]

output = pipe(msg, **generation_args)

print(output[0]['generated_text'])

 1. Paris, França

2. Roma, Itália

3. Londres, Reino Unido

4. Berlim, Alemanha

5. Madrid, Espanha

6. Lisboa, Portugal

7. Viena, Áustria

8. Amsterdã, Países Baixos

9. Budapeste, Hungria

10. Estocolmo, Suécia


In [8]:
# Configuração de Quantização para Otimização de Memória e Desempenho
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Habilita a quantização do modelo para 4 bits, reduzindo o uso de memória durante o carregamento.
    bnb_4bit_quant_type="nf4",  # Define o tipo de quantização como 'nf4' (Normal Float 4), otimizando precisão e desempenho.
    bnb_4bit_use_double_quant=True,  # Ativa a quantização dupla, aplicando uma segunda etapa de quantização para compactar ainda mais os pesos.
    bnb_4bit_compute_dtype=torch.float16  # Especifica o tipo de dado como float16 (meia precisão) para cálculos durante o uso do modelo, economizando memória e acelerando o processamento.
)

# model_id = "microsoft/Phi-3-mini-4k-instruct"
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# criando o tokenizador
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code = True # Permite o carregamento personalizado de modelos
    )

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=quantization_config,
    tokenizer = tokenizer
    )

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/75/15/75158e9667a787ec0ccbd8d18412c5d9a5f7326c543b803c4932dbc95951ed30/92ecfe1a2414458b4821ac8c13cf8cb70aed66b5eea8dc5ad9eeb4ff309d6d7b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00004-of-00004.safetensors%3B+filename%3D%22model-00004-of-00004.safetensors%22%3B&Expires=1736620164&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjYyMDE2NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc1LzE1Lzc1MTU4ZTk2NjdhNzg3ZWMwY2NiZDhkMTg0MTJjNWQ5YTVmNzMyNmM1NDNiODAzYzQ5MzJkYmM5NTk1MWVkMzAvOTJlY2ZlMWEyNDE0NDU4YjQ4MjFhYzhjMTNjZjhjYjcwYWVkNjZiNWVlYThkYzVhZDllZWI0ZmYzMDlkNmQ3Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=bJLboTgOVmb8fIb9Dohi707zK4DDVMc%7ElGYmjkLQZ67AslnKRHMaIgatvZS44xaK9nJxNmI%7EGufNHqmyo2cY06euRglwOjKOlATU0Zh2mktuXll32-wKOHMCdqyOI6tnGa11Z22Ve1TqEXE80P59bglwfS1zUAl-eulP9THsR9lVtyplMjDp-QIiNa5aXoPSoAtUE6x%7Emp4-yz1fXTt-22anDcEbmv1B

model-00004-of-00004.safetensors:  17%|#7        | 199M/1.17G [00:00<?, ?B/s]

TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'tokenizer'

In [ ]:
prompt = "Quem foi a primeira pessoa a ir pro espaço?"
messages = [{"role": "user", "content": prompt}]

In [ ]:
# Codifica as mensagens em tensores compatíveis com o modelo
encodeds = tokenizer(messages, return_tensors="pt")

# Move os dados codificados para o dispositivo (GPU ou CPU)
model_inputs = encodeds.to(device)

# Gera texto com base nos dados de entrada
generated_ids = model.generate(
    model_inputs,                      # Dados de entrada do modelo
    max_new_tokens=1000,               # Número máximo de tokens a serem gerados
    do_sample=True,                    # Ativa a amostragem para maior diversidade na geração
    pad_token_id=tokenizer.eos_token_id # Define o token de preenchimento como o token de fim de sequência
)

# Decodifica os tokens gerados em texto legível
decoded = tokenizer.batch_decode(generated_ids)

# Extrai o resultado decodificado
res = decoded[0]
res  # Retorna o texto gerado
